In [ ]:
import os
import xmlrpc.client
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
def api_params_func(test_db: bool = False) -> dict:

    api_url = os.environ.get('ODOO_URL_API')
    api_db = os.environ.get('ODOO_DB_API')
    api_test_db = os.environ.get('ODOO_DB_PRUEBA_API')
    api_username = os.environ.get('ODOO_USERNAME_API')
    api_clave = os.environ.get('ODOO_CLAVE_API')


    api_params = {}
    if test_db:
        api_params['api_db'] = api_test_db
    else:
        api_params['api_db'] = api_db


    common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
    uid = common.authenticate(api_params['api_db'], api_username, api_clave, {})
    models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')


    api_params['api_clave'] = api_clave
    api_params['api_uid'] = uid
    api_params['api_models'] = models

    return api_params

In [ ]:
def search_pay_func(mes: int) -> list[str]:
    
    if type(mes) != int or mes < 1 or mes > 12:
        raise Exception (f'El mes es incorrecto. El párametro "mes" debe ser un número entero entre 1 y 12. Escribiste: {mes}')
    

    param_dia_ini = datetime(2024, mes, 1)
    param_dia_fin = datetime(2024, mes + 1, 1) - timedelta(days= 1)

    search_pay_acc = [
        "&", "&", "&",
            ("partner_type", "=", "customer"),
            ("is_internal_transfer", "=", False),
            ("state", "=", "posted"),
        "&",
            ("date", ">=", param_dia_ini.strftime('%Y-%m-%d')),
            ("date", "<=", param_dia_fin.strftime('%Y-%m-%d')),
        ]


    param_dia_ini_pos = param_dia_ini + timedelta(hours=7)
    param_dia_fin_pos = param_dia_fin + timedelta(hours=31) - timedelta(seconds= 1)
    
    search_pay_pos = [
        "&",
            ("payment_date", ">=", param_dia_ini_pos.strftime('%Y-%m-%d %H:%M:%S')),
            ("payment_date", "<=", param_dia_fin_pos.strftime('%Y-%m-%d %H:%M:%S')),
        ]
    

    return search_pay_acc, search_pay_pos

In [ ]:
def api_call_pay_acc_func(api_params: dict, search_pay: list[str] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']


    search_pay_acc = search_pay[0]


    pay_acc_fields = [
                    'name',
                    'date',
                    'partner_id',
                    'amount',
                    'ref',
                    'reconciled_invoice_ids',
                    'pos_session_id'
                    ]

    pay_acc_ids = models.execute_kw(api_db, uid, api_clave, 'account.payment', 'search', [search_pay_acc])
    pay_acc_json = models.execute_kw(api_db, uid, api_clave, 'account.payment', 'read', [pay_acc_ids], {'fields': pay_acc_fields})

    
    return pay_acc_json

In [ ]:
def pay_acc_df_func(pay_acc_json: list[dict]) -> list[pd.DataFrame, int]:
    
    data_pay_acc = []
    data_fact_ids = []

    for pay in pay_acc_json:
        if not pay['pos_session_id']:
            new = {}
            new['id'] = pay['id']
            new['name'] = pay['name']
            new['date'] = pay['date']
            new['partner_id'] = pay['partner_id']
            new['amount'] = pay['amount']
            new['ref'] = pay['ref'] if pay['ref'] else pd.NA
            new['pay_fact_docs'] = pay['reconciled_invoice_ids'] if pay['reconciled_invoice_ids'] else pd.NA
            
            data_fact_ids += pay['reconciled_invoice_ids']

            data_pay_acc.append(new)

    pay_acc_df = pd.DataFrame(data_pay_acc)


    pay_acc_df['date'] = pd.to_datetime(pay_acc_df['date'], format='%Y-%m-%d')
    
    return pay_acc_df, data_fact_ids

In [ ]:
def api_call_pay_pos_func(api_params: dict, search_pay: list[str] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']


    search_pay_pos = search_pay[1]


    pay_pos_fields = [
                    'pos_order_id',
                    'payment_date',
                    'amount',
                    'payment_method_id'
                    ]

    pay_pos_ids = models.execute_kw(api_db, uid, api_clave, 'pos.payment', 'search', [search_pay_pos])
    pay_pos_json = models.execute_kw(api_db, uid, api_clave, 'pos.payment', 'read', [pay_pos_ids], {'fields': pay_pos_fields})
    

    return pay_pos_json

In [ ]:
def pay_pos_df_func(pay_pos_json: list[dict]) -> list[pd.DataFrame, int]:
    
    data_pay_pos = []
    data_pos_ids = set()

    for pay in pay_pos_json:
        new = {}
        new['pay_pos_id'] = pay['id']
        new['pos_order_id'] = pay['pos_order_id'][0]
        new['pos_order_name'] = pay['pos_order_id'][1]
        new['date'] = pay['payment_date']
        new['amount'] = pay['amount']
        new['payment_method'] = pay['payment_method_id'][1]
    
        data_pos_ids.add(pay['pos_order_id'][0])
        data_pay_pos.append(new)

    pay_pos_df = pd.DataFrame(data_pay_pos)

    pay_pos_df['date'] = pd.to_datetime(pay_pos_df['date'], format='%Y-%m-%d %H:%M:%S')
    
    return pay_pos_df, list(data_pos_ids)

In [ ]:
def api_call_pos_order_func(api_params: dict, data_pos_ids: list[int] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']

    pos_order_fields = [
                    'name',
                    'account_move',
                    'partner_id',
                    'session_id',
                    'amount_total'
                    ]

    pos_order_json = models.execute_kw(api_db, uid, api_clave, 'pos.order', 'read', [data_pos_ids], {'fields': pos_order_fields})

    return pos_order_json

In [ ]:
def pos_order_func(pos_order_json: list[dict], data_fact_ids: list[int]) -> list[list[int]]:
    
    data_pos_amount_cero = [] 
  
    for pos in pos_order_json:
        
        if pos['account_move']:
            data_fact_ids.append(pos['account_move'][0])

        if pos['amount_total'] == 0:
            data_pos_amount_cero.append(pos)


    pos_amount_cero_df = pd.DataFrame(data_pos_amount_cero)
    
    if not pos_amount_cero_df.empty:
        pos_amount_cero_df['session_id'] = pos_amount_cero_df['session_id'].str.get(1)
        
    
    return data_fact_ids, pos_amount_cero_df

In [ ]:
def api_call_acccount_docs_func(api_params: dict, data_fact_ids: list[int] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']

    data_fact_fields = [
                    'name',
                    'partner_id',
                    'date',
                    'invoice_payments_widget',
                    'amount_total',
                    'amount_residual'
                    ]

    fact_doc_json = models.execute_kw(api_db, uid, api_clave, 'account.move', 'read', [data_fact_ids], {'fields': data_fact_fields})

    return fact_doc_json

In [ ]:
def pay_fact_df_func(mes: int, fact_doc_json: dict) -> pd.DataFrame:
    
    data_pay_fact = []

    for fact in fact_doc_json:
        if fact['invoice_payments_widget']:
            for pay in fact['invoice_payments_widget']['content']:
                if 'Facturas' not in pay['journal_name'] and datetime.strptime(pay['date'], '%Y-%m-%d').month == mes:
                    new = {}
                    new['fac_doc_id'] = fact['id']
                    new['fac_doc_name'] = fact['name']
                    new['fac_doc_cliente'] = fact['partner_id'][1]
                    new['fac_doc_date'] = fact['date']
                    new['fac_doc_total'] = fact['amount_total']
                    new['fac_doc_deuda'] = fact['amount_residual']
                    new['pay_journal'] = pay['journal_name']
                    new['pay_amount'] = pay['amount']
                    new['pay_date'] = pay['date']
                    new['pay_pos'] = pay['pos_payment_name'] if pay['pos_payment_name'] else pd.NA
        
                    data_pay_fact.append(new)

    pay_fact_df = pd.DataFrame(data_pay_fact)
    pay_fact_df['fac_doc_date'] = pd.to_datetime(pay_fact_df['fac_doc_date'], format='%Y-%m-%d')
    pay_fact_df['pay_date'] = pd.to_datetime(pay_fact_df['pay_date'], format='%Y-%m-%d')

    return pay_fact_df

In [ ]:
def pagos_faltantes_check_func(pay_acc_df: pd.DataFrame, pos_amount_cero_df: pd.DataFrame, pay_pos_df: pd.DataFrame) -> bool:
    
    pagos_faltantes = []

    if not pos_amount_cero_df.empty:
        acc_cobranza_pdv_df = pay_acc_df[(~pay_acc_df['ref'].isna()) & (pay_acc_df['ref'].str[:1] != 'F')]
        pdv_cobranza_df = pos_amount_cero_df.merge(pay_pos_df, how='left', left_on='id', right_on='pos_order_id')

        for i in range(len(pdv_cobranza_df)):
            mini_df = acc_cobranza_pdv_df.loc[
                (acc_cobranza_pdv_df['ref'].str.contains(pdv_cobranza_df['name'].iloc[i])) 
                | (acc_cobranza_pdv_df['ref'].str.contains(pdv_cobranza_df['session_id'].iloc[i])) 
                & (acc_cobranza_pdv_df['amount'] == pdv_cobranza_df['amount'].iloc[i])
                & (acc_cobranza_pdv_df['partner_id'].str.get(0) == pdv_cobranza_df['partner_id'].iloc[i][0])]

            if mini_df.empty:
                new = {}
                new['pos_doc_id'] = pdv_cobranza_df.iloc[i]['id']
                new['pos_doc_name'] = pdv_cobranza_df.iloc[i]['name']
                new['cliente'] = pdv_cobranza_df.iloc[i]['partner_id'][1]
                new['monto'] = pdv_cobranza_df.iloc[i]['amount']
                new['fecha'] = pdv_cobranza_df.iloc[i]['date']- timedelta(hours=7)

                pagos_faltantes.append(new)

    pagos_faltantes_df = pd.DataFrame(pagos_faltantes)

    if pagos_faltantes_df.empty:
        return True

    pagos_faltantes_df.to_excel('Cobranza_PdV_Faltante.xlsx')

In [46]:
mes = 4

api_params = api_params_func(True)
search_pay = search_pay_func(mes)

pay_acc_json = api_call_pay_acc_func(api_params, search_pay)
pay_acc_df, data_fact_ids2 = pay_acc_df_func(pay_acc_json)

pay_pos_json = api_call_pay_pos_func(api_params, search_pay)
pay_pos_df, data_pos_ids = pay_pos_df_func(pay_pos_json)

pos_order_json = api_call_pos_order_func(api_params, data_pos_ids)
data_fact_ids, pos_amount_cero_df = pos_order_func(pos_order_json, data_fact_ids2)

pagos_faltantes_check = pagos_faltantes_check_func(pay_acc_df, pos_amount_cero_df, pay_pos_df)

if pagos_faltantes_check:
    fact_doc_json = api_call_acccount_docs_func(api_params, data_fact_ids)
    pay_fact_df = pay_fact_df_func(mes, fact_doc_json)

else:
    print('Corrige los pagos PdV faltantes')

## Algoritmo de comparación

In [67]:
pay_pos_df['date_day'] = pay_pos_df['date'].dt.date
pay_pos_df['date'] = pay_pos_df['date'].dt.strftime('%d-%m-%Y')
pay_pos_df.loc[pay_pos_df['payment_method'] == 'Tarjeta Débito A1']

,pay_pos_id,pos_order_id,pos_order_name,date,amount,payment_method,date_day
14,21821,15313,PdV CSL/6465,01-05-2024,511.35,Tarjeta Débito A1,2024-05-01
25,21810,15306,PdV CSL/6460,30-04-2024,"1,988.39",Tarjeta Débito A1,2024-04-30
48,21787,15289,PdV CSL/6452,30-04-2024,556.87,Tarjeta Débito A1,2024-04-30
51,21784,15287,PdV CSL/6451,30-04-2024,238.45,Tarjeta Débito A1,2024-04-30
63,21772,15279,PdV CSL/6448,30-04-2024,334.30,Tarjeta Débito A1,2024-04-30
...,...,...,...,...,...,...,...
5860,15972,11269,PdV CSL/4637,01-04-2024,574.56,Tarjeta Débito A1,2024-04-01
5878,15954,11256,PdV CSL/4634,01-04-2024,50.34,Tarjeta Débito A1,2024-04-01
5883,15949,11252,PdV CSL/4632,01-04-2024,56.96,Tarjeta Débito A1,2024-04-01
5887,15945,11249,PdV CSL/4630,01-04-2024,145.43,Tarjeta Débito A1,2024-04-01


In [68]:
pay_fact_df['date'] = pay_fact_df['pay_date'].dt.strftime('%d-%m-%Y')

# pay_fact_df.loc[pay_fact_df['pay_pos'] == 'Tarjeta Débito A1'].to_excel('Tarjeta Débito A1.xlsx')
pay_fact_df.loc[pay_fact_df['pay_pos'] == 'Tarjeta Débito A1']

,fac_doc_id,fac_doc_name,fac_doc_cliente,fac_doc_date,fac_doc_total,fac_doc_deuda,pay_journal,pay_amount,pay_date,pay_pos,date
761,52474,F1-CC/2024/04672,Mostrador,2024-04-01,437.87,0.00,Punto De Venta A1,437.87,2024-04-01,Tarjeta Débito A1,01-04-2024
767,52505,F1-CC/2024/04673,ANA GABRIELA RIVERA GONZALEZ,2024-04-01,145.43,0.00,Punto De Venta A1,145.43,2024-04-01,Tarjeta Débito A1,01-04-2024
770,52516,F1-CC/2024/04675,Mostrador,2024-04-01,56.96,0.00,Punto De Venta A1,56.96,2024-04-01,Tarjeta Débito A1,01-04-2024
774,52529,F1-CC/2024/04677,DMADERA BCS,2024-04-01,50.34,0.00,Punto De Venta A1,50.34,2024-04-01,Tarjeta Débito A1,01-04-2024
787,52585,F1-CC/2024/04680,PUERTAS Y DISEÑOS DE MADERAS,2024-04-01,574.56,0.00,Punto De Venta A1,574.56,2024-04-01,Tarjeta Débito A1,01-04-2024
...,...,...,...,...,...,...,...,...,...,...,...
4519,72412,F1-CC/2024/06486,Mostrador,2024-04-30,334.30,0.00,Punto De Venta A1,334.30,2024-04-30,Tarjeta Débito A1,30-04-2024
4526,72447,F1-CC/2024/06489,Mostrador,2024-04-30,238.45,0.00,Punto De Venta A1,238.45,2024-04-30,Tarjeta Débito A1,30-04-2024
4528,72454,F1-CC/2024/06490,MARIA GARCIA GUTIERREZ,2024-04-30,556.87,0.00,Punto De Venta A1,556.87,2024-04-30,Tarjeta Débito A1,30-04-2024
4545,72522,F1-CC/2024/06498,AVANTEC LOS CABOS,2024-04-30,"1,988.39",0.00,Punto De Venta A1,"1,988.39",2024-04-30,Tarjeta Débito A1,30-04-2024


In [69]:
wep = pd.merge(pay_pos_df.loc[pay_pos_df['payment_method'] == 'Tarjeta Débito A1'], pay_fact_df.loc[pay_fact_df['pay_pos'] == 'Tarjeta Débito A1'], how='outer', left_on=['amount',	'date'], right_on=['pay_amount', 'date'])

In [75]:
wep

,pay_pos_id,pos_order_id,pos_order_name,date,amount,payment_method,date_day,fac_doc_id,fac_doc_name,fac_doc_cliente,fac_doc_date,fac_doc_total,fac_doc_deuda,pay_journal,pay_amount,pay_date,pay_pos
0,"20,361.00","14,310.00",PdV CSL/6003,23-04-2024,6.86,Tarjeta Débito A1,2024-04-23,"67,626.00",F1-CC/2024/06038,RECOLECTORA DE LA CIUDAD,2024-04-23,6.86,0.00,Punto De Venta A1,6.86,2024-04-23,Tarjeta Débito A1
1,"17,239.00","12,159.00",PdV CSL/5028,08-04-2024,17.67,Tarjeta Débito A1,2024-04-08,"56,888.00",F1-CC/2024/05072,Mostrador,2024-04-08,17.67,0.00,Punto De Venta A1,17.67,2024-04-08,Tarjeta Débito A1
2,"17,037.00","12,023.00",PdV CSL/4963,06-04-2024,26.01,Tarjeta Débito A1,2024-04-06,"56,240.00",F1-CC/2024/05002,LIFESTYLE MANAGEMENT SERVICES,2024-04-06,26.01,0.00,Punto De Venta A1,26.01,2024-04-06,Tarjeta Débito A1
3,"19,202.00","13,487.00",PdV CSL/5632,17-04-2024,27.99,Tarjeta Débito A1,2024-04-17,"63,822.00",F1-CC/2024/05668,Mostrador,2024-04-17,27.99,0.00,Punto De Venta A1,27.99,2024-04-17,Tarjeta Débito A1
4,"16,391.00","11,578.00",PdV CSL/4763,03-04-2024,35.95,Tarjeta Débito A1,2024-04-03,"54,214.00",F1-CC/2024/04806,Mostrador,2024-04-03,35.95,0.00,Punto De Venta A1,35.95,2024-04-03,Tarjeta Débito A1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,"20,027.00","14,063.00",PdV CSL/5888,22-04-2024,"8,187.76",Tarjeta Débito A1,2024-04-22,"66,476.00",F1-CC/2024/05923,CARLOS ALBERTO MARTINEZ COTA,2024-04-22,"8,187.76",0.00,Punto De Venta A1,"8,187.76",2024-04-22,Tarjeta Débito A1
489,"19,834.00","13,927.00",PdV CSL/5833,20-04-2024,"15,000.00",Tarjeta Débito A1,2024-04-20,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN
490,"20,308.00","14,270.00",PdV CSL/5979,23-04-2024,"15,606.14",Tarjeta Débito A1,2024-04-23,"67,485.00",F1-CC/2024/06015,AVANTEC LOS CABOS,2024-04-23,"15,606.14",0.00,Punto De Venta A1,"15,606.14",2024-04-23,Tarjeta Débito A1
491,"21,562.00","15,137.00",PdV CSL/6379,29-04-2024,"16,813.69",Tarjeta Débito A1,2024-04-29,"71,716.00",F1-CC/2024/06419,MARLENE HERRERA ALAVEZ,2024-04-29,"16,813.69",0.00,Punto De Venta A1,"16,813.69",2024-04-29,Tarjeta Débito A1


## Pruebas

In [50]:
pay_pos_df[['payment_method', 'amount']].groupby('payment_method').agg({'amount': ['count', 'sum']})

amount           
                                     count        sum
payment_method                                       
Crédito del cliente                    365 664,202.16
Efectivo A1                           1912 445,517.16
Efectivo A2                           2116 427,838.29
Tarjeta Crédito A1                     140 149,936.21
Tarjeta Crédito A2                     223 305,941.02
Tarjeta Débito A1                      464 477,546.77
Tarjeta Débito A2                      524 500,456.98
Transferencia Banco Sant92000702524    150 810,840.96
Transferencia Banco Scot23900002860      6   6,510.51

In [59]:
acc_cobranza_pdv_df = pay_acc_df[(~pay_acc_df['ref'].isna()) & (pay_acc_df['ref'].str[:1] != 'F')]

if not pos_amount_cero_df.empty:

    pdv_cobranza_df = pos_amount_cero_df.merge(pay_pos_df, how='left', left_on='id', right_on='pos_order_id')
    pdv_cobranza_df[['payment_method', 'amount']].groupby('payment_method').agg({'amount': ['count', 'sum']})
    
else:
    print('No hay cobranza en PdV')

pdv_cobranza_df[['payment_method', 'amount']].groupby('payment_method').agg({'amount': ['count', 'sum']})

amount            
                     count         sum
payment_method                        
Crédito del cliente     21 -105,799.84
Efectivo A1             15   66,118.67
Efectivo A2              4    2,745.57
Tarjeta Débito A1        2   36,935.60

In [60]:
pay_fact_df[['pay_pos', 'pay_amount']].groupby('pay_pos').agg({'pay_amount': ['count', 'sum']})

pay_amount           
                                         count        sum
pay_pos                                                  
Efectivo A1                               1037 375,753.88
Efectivo A2                               1196 424,787.38
Tarjeta Crédito A1                         136 149,800.80
Tarjeta Crédito A2                         220 305,872.22
Tarjeta Débito A1                          459 436,223.76
Tarjeta Débito A2                          523 507,382.38
Transferencia Banco Sant92000702524        146 810,837.28
Transferencia Banco Scot23900002860          6   6,510.51

In [61]:
# pay_fact_df.loc[pay_fact_df['pay_pos'] == 'Efectivo A1'].to_excel('Efectivo_A1.xlsx')
pay_fact_df.loc[pay_fact_df['pay_pos'] == 'Efectivo A1']

,fac_doc_id,fac_doc_name,fac_doc_cliente,fac_doc_date,fac_doc_total,fac_doc_deuda,pay_journal,pay_amount,pay_date,pay_pos
769,52511,F1-CC/2024/04674,CRUZ DOMINGO CHAVEZ CAMACHO,2024-04-01,964.11,0.00,Punto De Venta A1,964.11,2024-04-01,Efectivo A1
771,52519,F1-CC/2024/04676,Mostrador,2024-04-01,35.35,0.00,Punto De Venta A1,35.35,2024-04-01,Efectivo A1
783,52567,F1-CC/2024/04679,Mostrador,2024-04-01,86.90,0.00,Punto De Venta A1,86.90,2024-04-01,Efectivo A1
789,52592,F1-CC/2024/04681,Mostrador,2024-04-01,364.07,0.00,Punto De Venta A1,364.07,2024-04-01,Efectivo A1
790,52595,F1-CC/2024/04682,Abiel Ceron Moreno,2024-04-01,"1,820.78",0.00,Punto De Venta A1,"1,820.78",2024-04-01,Efectivo A1
...,...,...,...,...,...,...,...,...,...,...
4538,72488,F1-CC/2024/06496,Emilio Costich Lopez,2024-04-30,169.43,0.00,Punto De Venta A1,169.43,2024-04-30,Efectivo A1
4542,72511,F1-CC/2024/06497,Emilio Costich Lopez,2024-04-30,134.85,0.00,Punto De Venta A1,134.85,2024-04-30,Efectivo A1
4546,72525,F1-CC/2024/06499,Emilio Costich Lopez,2024-04-30,164.28,0.00,Punto De Venta A1,164.28,2024-04-30,Efectivo A1
4547,72532,F1-CC/2024/06500,Emilio Costich Lopez,2024-04-30,289.21,0.00,Punto De Venta A1,289.21,2024-04-30,Efectivo A1


In [62]:
pay_fact_df[['pay_journal', 'pay_amount']].groupby('pay_journal').agg({'pay_amount': ['count', 'sum']})

pay_amount             
                           count          sum
pay_journal                                  
Banco Bana70103464895         11   116,869.17
Banco Sant65507231316        555 3,426,051.23
Banco Sant92000702524        186   471,485.21
Efectivo A1                   67    86,233.39
Efectivo A2                    5     4,745.57
Punto De Venta A1           1683 1,138,104.84
Punto De Venta A2           2040 1,879,063.37
Tarjeta Deb/Cré A1            13    36,935.60

In [63]:
pay_fact_df.agg({'pay_amount':['sum']})

,pay_amount
sum,"7,159,488.38"
